In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

class PdScorer:
    def __init__(self, fitted_model):
        self.model = fitted_model
        self.grid_scores = self.create_grid_scores()

    def create_grid_scores(self):
        # Extraction des coefficients estimés du modèle
        coefficients = self.model.params

        # Normalisation des coefficients pour les rendre comparables
        normalized_coefficients = (coefficients - coefficients.min()) / (coefficients.max() - coefficients.min())

        # Création d'une grille de score sur mille basée sur les coefficients normalisés
        grid_scores = np.round(normalized_coefficients * 1000).astype(int)

        return grid_scores

    def get_grid_scores(self):
        return self.grid_scores

    def transform(self, data, target_col):
        # Ajoute une colonne constante pour intercept dans data
        data = sm.add_constant(data)

        # Extraction de la variable cible (target)
        y = data[target_col]

        # Suppression de la variable cible du DataFrame
        data = data.drop(columns=[target_col])

        # Calcul des scores normalisés sur mille en utilisant les coefficients normalisés
        normalized_scores = np.dot(data, self.grid_scores[1:]) + self.grid_scores[0]

        # Transformation des scores normalisés en scores sur 1000
        scores_on_thousand = np.round((normalized_scores - normalized_scores.min()) / (normalized_scores.max() - normalized_scores.min()) * 1000).astype(int)

        return scores_on_thousand



In [18]:
# Exemple d'utilisation
# Génération de données d'exemple (remplacez X et y par vos propres données)
np.random.seed(42)
data1 = pd.DataFrame({
    'Category1': np.random.choice(['A', 'B', 'C'], size=100),
    'Category2': np.random.choice(['X', 'Y', 'Z'], size=100),
    'Variable1': np.random.rand(100),
    'Variable2': np.random.rand(100),
    'Target': np.random.choice([0, 1], size=100),
})

data2 = pd.DataFrame({
    'Category1': np.random.choice(['A', 'B', 'C'], size=50),
    'Category2': np.random.choice(['X', 'Y', 'Z'], size=50),
    'Variable1': np.random.rand(50),
    'Variable2': np.random.rand(50),
    'Target': np.random.choice([0, 1], size=50),
})

# Spécification de la formule pour le modèle logistique
formula = 'Target ~ Category1 + Category2 + Variable1 + Variable2'

# Estimation du modèle de régression logistique avec statsmodels
logit_model = sm.Logit.from_formula(formula, data1)
fitted_model = logit_model.fit(disp=False)

In [19]:
print(fitted_model.summary())

                           Logit Regression Results                           
Dep. Variable:                 Target   No. Observations:                  100
Model:                          Logit   Df Residuals:                       93
Method:                           MLE   Df Model:                            6
Date:                Sat, 24 Feb 2024   Pseudo R-squ.:                 0.04872
Time:                        09:41:52   Log-Likelihood:                -65.633
converged:                       True   LL-Null:                       -68.994
Covariance Type:            nonrobust   LLR p-value:                    0.3472
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.9446      0.670      1.410      0.159      -0.369       2.258
Category1[T.B]    -0.2421      0.505     -0.479      0.632      -1.232       0.748
Category1[T.C]     0.4621      0.528

In [8]:
# Instanciation de la classe PdScorer avec le modèle ajusté
pd_scorer = PdScorer(fitted_model)

# Utilisation de la méthode get_grid_scores pour obtenir la grille de score
grid_scores = pd_scorer.get_grid_scores()
print("Grille de score sur mille:")
print(grid_scores)

# Utilisation de la méthode transform pour obtenir les scores sur 1000 pour data1
# Assurez-vous que les noms de colonnes dans 'data1' correspondent à ceux utilisés dans le modèle
scores_data1 = pd_scorer.transform(data1, 'Target')
# Affichage des 10 premiers scores pour data1
print("\nScores sur 1000 pour les 10 premières lignes de data1:")
print(scores_data1[:10])

# Utilisation de la méthode transform pour obtenir les scores sur 1000 pour data2
# Assurez-vous que les noms de colonnes dans 'data2' correspondent à ceux utilisés dans le modèle
scores_data2 = pd_scorer.transform(data2, 'Target')
# Affichage des 10 premiers scores pour data2
print("\nScores sur 1000 pour les 10 premières lignes de data2:")
print(scores_data2[:10])

Grille de score sur mille:
Intercept         1000
Category1[T.B]     388
Category1[T.C]     751
Category2[T.Y]     433
Category2[T.Z]     154
Variable1            0
Variable2          568
dtype: int32


ValueError: shapes (100,5) and (6,) not aligned: 5 (dim 1) != 6 (dim 0)

In [21]:
np.random.seed(42)
data1 = pd.DataFrame({
    'Category1': np.random.choice(['A', 'B', 'C'], size=100),
    'Category2': np.random.choice(['X', 'Y', 'Z'], size=100),
    'Variable1': np.random.rand(100),
    'Variable2': np.random.rand(100),
    'Target': np.random.choice([0, 1], size=100),
})
formula = 'Target ~ Category1 + Category2 + Variable1 + Variable2'

# Estimation du modèle de régression logistique avec statsmodels
logit_model = sm.Logit.from_formula(formula, data1)
fitted_model = logit_model.fit(disp=False)

In [24]:
print(fitted_model.summary())

                           Logit Regression Results                           
Dep. Variable:                 Target   No. Observations:                  100
Model:                          Logit   Df Residuals:                       93
Method:                           MLE   Df Model:                            6
Date:                Sat, 24 Feb 2024   Pseudo R-squ.:                 0.04872
Time:                        09:44:43   Log-Likelihood:                -65.633
converged:                       True   LL-Null:                       -68.994
Covariance Type:            nonrobust   LLR p-value:                    0.3472
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.9446      0.670      1.410      0.159      -0.369       2.258
Category1[T.B]    -0.2421      0.505     -0.479      0.632      -1.232       0.748
Category1[T.C]     0.4621      0.528

In [25]:
data1

,Category1,Category2,Variable1,Variable2,Target
0,C,Z,0.403836,0.768554,0
1,A,Z,0.064892,0.043604,1
2,C,Z,0.253915,0.994551,1
3,C,X,0.246876,0.469945,1
4,A,Z,0.696304,0.279560,1
...,...,...,...,...,...
95,A,Y,0.668924,0.341880,0
96,A,Z,0.864168,0.091799,0
97,C,Z,0.230185,0.094157,0
98,A,Y,0.499193,0.311413,1


In [29]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

np.random.seed(42)
data1 = pd.DataFrame({
    #'Category1': np.random.choice(['A', 'B', 'C'], size=100),
    #'Category2': np.random.choice(['X', 'Y', 'Z'], size=100),
    'Variable1': np.random.rand(100),
    'Variable2': np.random.rand(100),
    'Target': np.random.choice([0, 1], size=100),
})
formula = 'Target ~  Variable1 + Variable2'

# Estimation du modèle de régression logistique avec statsmodels
logit_model = sm.Logit.from_formula(formula, data1)
fitted_model = logit_model.fit(disp=False)


# Classe pour la grille de score
class PdScorer:
    def __init__(self, fitted_model, data, target_col):
        self.model = fitted_model
        self.data = data
        self.target_col = target_col
        self.grid_scores = self.create_grid_scores()

    def create_grid_scores(self):
        # Extraction des coefficients estimés du modèle
        coefficients = self.model.params

        # Vérifier si 'Intercept' est présent dans les coefficients
        if 'Intercept' in coefficients.index:
            # Exclure 'Intercept' de la grille de score
            coefficients = coefficients.drop('Intercept')

        # Normalisation des coefficients pour les rendre comparables
        normalized_coefficients = (coefficients - coefficients.min()) / (coefficients.max() - coefficients.min())

        # Création d'une grille de score sur mille basée sur les coefficients normalisés
        grid_scores = np.round(normalized_coefficients * 1000).astype(int)

        # Création d'un DataFrame pour stocker les informations de chaque classe
        result_df = pd.DataFrame(columns=['Variable', 'Classe', 'P-Value', 'Note', 'Contribution', 'Taux_de_Defaut', 'Effectif'])

        # Boucle sur les variables explicatives
        for variable in normalized_coefficients.index:
            # Extraction des variables explicatives
            X = self.data[variable] if 'const' in self.data.columns else sm.add_constant(self.data[variable])


            # Prédiction des probabilités
            y_prob = self.model.predict(X)

            # Création de classes basées sur les quantiles
            classes = pd.qcut(y_prob, q=[0, 0.2, 0.4, 0.6, 0.8, 1.0], labels=False)

            # Calcul des statistiques pour chaque classe
            for classe in range(classes.nunique()):
                indices = (classes == classe)
                p_value = sm.stats.proportions_ztest(self.data.loc[indices, self.target_col].sum(), indices.sum())[1]
                contribution = coefficients[variable] * (self.data.loc[indices, variable].mean() - self.data[variable].mean())
                taux_defaut = self.data.loc[indices, self.target_col].mean()
                effectif = indices.sum()

                # Ajout des informations à result_df
                result_df = result_df.append({
                    'Variable': variable,
                    'Classe': classe,
                    'P-Value': p_value,
                    'Note': grid_scores[variable],
                    'Contribution': contribution,
                    'Taux_de_Defaut': taux_defaut,
                    'Effectif': effectif
                }, ignore_index=True)

        return result_df

    def get_grid_scores(self):
        return self.grid_scores

# Utilisation de la classe PdScorer
scorer = PdScorer(fitted_model, data1, 'TARGET')
grid_scores_df = scorer.get_grid_scores()

# Affichage de la grille de score
print(grid_scores_df)



PatsyError: predict requires that you use a DataFrame when predicting from a model
that was created using the formula api.

The original error message returned by patsy is:
Error evaluating factor: NameError: name 'Variable2' is not defined
    Target ~  Variable1 + Variable2
                          ^^^^^^^^^

In [27]:
# Utilisation de la classe PdScorer
scorer = PdScorer(fitted_model, data1, 'TARGET')
grid_scores_df = scorer.get_grid_scores()

# Affichage de la grille de score
print(grid_scores_df)

KeyError: 'Category1[T.B]'

In [34]:
# AVec var catégorielle 

np.random.seed(42)
data1 = pd.DataFrame({
    'Category1': np.random.choice(['A', 'B', 'C'], size=100),
    'Category2': np.random.choice(['X', 'Y', 'Z'], size=100),
    'Variable1': np.random.rand(100),
    'Variable2': np.random.rand(100),
    'Target': np.random.choice([0, 1], size=100),
})
formula = 'Target ~ Category1 + Category2 + Variable1 + Variable2'

# Estimation du modèle de régression logistique avec statsmodels
logit_model = sm.Logit.from_formula(formula, data1)
fitted_model = logit_model.fit(disp=False)


class PdScorer:
    def __init__(self, fitted_model, data, target_col):
        self.model = fitted_model
        self.data = data
        self.target_col = target_col
        self.grid_scores = self.create_grid_scores()

    def create_grid_scores(self):
        # Extraction des coefficients estimés du modèle
        coefficients = self.model.params

        # Vérifier si 'Intercept' est présent dans les coefficients
        if 'Intercept' in coefficients.index:
            # Exclure 'Intercept' de la grille de score
            coefficients = coefficients.drop('Intercept')

        # Normalisation des coefficients pour les rendre comparables
        normalized_coefficients = (coefficients - coefficients.min()) / (coefficients.max() - coefficients.min())

        # Création d'une grille de score sur mille basée sur les coefficients normalisés
        grid_scores = np.round(normalized_coefficients * 1000).astype(int)



        # Création d'un DataFrame pour stocker les informations de chaque classe
        result_df = pd.DataFrame(columns=['Variable', 'Classe', 'P-Value', 'Note', 'Contribution', 'Taux_de_Defaut', 'Effectif'])

        # Boucle sur les variables explicatives
        for variable in normalized_coefficients.index:
    # Extraire le nom de la variable sans le préfixe de catégorie
            var_name = variable.split('[')[0].strip()

    # Extraction des variables explicatives
            if var_name in self.data.columns:
                X = self.data[var_name] if 'const' in self.data.columns else sm.add_constant(self.data[var_name])


            # Prédiction des probabilités
            y_prob = self.model.predict(X)

            # Création de classes basées sur les quantiles
            classes = pd.qcut(y_prob, q=[0, 0.2, 0.4, 0.6, 0.8, 1.0], labels=False)

            # Calcul des statistiques pour chaque classe
            for classe in range(classes.nunique()):
                indices = (classes == classe)
                p_value = sm.stats.proportions_ztest(self.data.loc[indices, self.target_col].sum(), indices.sum())[1]
                contribution = coefficients[variable] * (self.data.loc[indices, var_name].mean() - self.data[var_name].mean())
                taux_defaut = self.data.loc[indices, self.target_col].mean()
                effectif = indices.sum()

                # Ajout des informations à result_df
                result_df = result_df.append({
                    'Variable': var_name,
                    'Classe': classe,
                    'P-Value': p_value,
                    'Note': grid_scores[variable],
                    'Contribution': contribution,
                    'Taux_de_Defaut': taux_defaut,
                    'Effectif': effectif
                }, ignore_index=True)

        return result_df

    def get_grid_scores(self):
        return self.grid_scores

# Utilisation de la classe PdScorer
scorer = PdScorer(logit_model, data1, 'Target')
grid_scores_df = scorer.get_grid_scores()

# Affichage de la grille de score
print(grid_scores_df)



AttributeError: 'Logit' object has no attribute 'params'